In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as xp
pd.set_option('display.max_rows', None)
import datetime
from plotly.subplots import make_subplots
import requests
from bs4 import BeautifulSoup

In [3]:
data = pd.read_csv('covid_19_data.csv')

In [4]:
data.head()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0


In [5]:
nul = [(c, data[c].isna().mean()*100) for c in data]
nul = pd.DataFrame(nul,columns=["columns name" , "percentage"])

In [6]:
nul

,columns name,percentage
0,SNo,0.000000
1,ObservationDate,0.000000
2,Province/State,40.970995
3,Country/Region,0.000000
4,Last Update,0.000000
5,Confirmed,0.000000
6,Deaths,0.000000
7,Recovered,0.000000


In [7]:
data["Province/State"]= data["Province/State"].fillna("unknown")

In [8]:
nul

,columns name,percentage
0,SNo,0.000000
1,ObservationDate,0.000000
2,Province/State,40.970995
3,Country/Region,0.000000
4,Last Update,0.000000
5,Confirmed,0.000000
6,Deaths,0.000000
7,Recovered,0.000000


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48095 entries, 0 to 48094
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   SNo              48095 non-null  int64  
 1   ObservationDate  48095 non-null  object 
 2   Province/State   48095 non-null  object 
 3   Country/Region   48095 non-null  object 
 4   Last Update      48095 non-null  object 
 5   Confirmed        48095 non-null  float64
 6   Deaths           48095 non-null  float64
 7   Recovered        48095 non-null  float64
dtypes: float64(3), int64(1), object(4)
memory usage: 2.9+ MB


In [10]:
data[["Confirmed", "Deaths" , "Recovered"]] = data[["Confirmed", "Deaths" , "Recovered"]].astype(int)

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48095 entries, 0 to 48094
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   SNo              48095 non-null  int64 
 1   ObservationDate  48095 non-null  object
 2   Province/State   48095 non-null  object
 3   Country/Region   48095 non-null  object
 4   Last Update      48095 non-null  object
 5   Confirmed        48095 non-null  int32 
 6   Deaths           48095 non-null  int32 
 7   Recovered        48095 non-null  int32 
dtypes: int32(3), int64(1), object(4)
memory usage: 2.4+ MB


In [12]:
data['Country/Region'] = data['Country/Region'] .replace("Mainland China" , "China")

In [13]:
data['Active_case'] = data['Confirmed'] - data['Deaths'] - data['Recovered']
data.head()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Active_case
0,1,01/22/2020,Anhui,China,1/22/2020 17:00,1,0,0,1
1,2,01/22/2020,Beijing,China,1/22/2020 17:00,14,0,0,14
2,3,01/22/2020,Chongqing,China,1/22/2020 17:00,6,0,0,6
3,4,01/22/2020,Fujian,China,1/22/2020 17:00,1,0,0,1
4,5,01/22/2020,Gansu,China,1/22/2020 17:00,0,0,0,0


In [14]:
Data = data[data['ObservationDate'] == max(data['ObservationDate'])].reset_index()

In [15]:
data.head()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Active_case
0,1,01/22/2020,Anhui,China,1/22/2020 17:00,1,0,0,1
1,2,01/22/2020,Beijing,China,1/22/2020 17:00,14,0,0,14
2,3,01/22/2020,Chongqing,China,1/22/2020 17:00,6,0,0,6
3,4,01/22/2020,Fujian,China,1/22/2020 17:00,1,0,0,1
4,5,01/22/2020,Gansu,China,1/22/2020 17:00,0,0,0,0


In [16]:

data.to_csv('data.csv')

In [17]:
Data_World= data.groupby(["ObservationDate"])[["Confirmed","Active_case","Recovered","Deaths"]].sum().reset_index()

In [18]:
Data_World.head()

,ObservationDate,Confirmed,Active_case,Recovered,Deaths
0,01/22/2020,555,510,28,17
1,01/23/2020,653,605,30,18
2,01/24/2020,941,879,36,26
3,01/25/2020,1438,1357,39,42
4,01/26/2020,2118,2010,52,56


In [19]:
labels = ["Last update","Confirmed","Active cases","Recoverd","Deathes"]
fig    =go.Figure(data=[go.Table(header=dict(values=labels),
                 cells=dict(values=Data_World.loc[0,["ObservationDate","Confirmed","Active_case","Recovered","Deaths"]]))
                     ])

fig.update_layout(
    title="Last update Corona in world")
fig.show()

In [20]:
labels = ["Active cases", "Recoverd","Deaths"]
values = Data_World.loc[0,["Active_case","Recovered","Deaths"]]

fig = xp.pie(Data_World, values=values, names=labels, color_discrete_sequence=['rgb(0,0,255)','rgb(77,150,33)','rgb(255,0,0)'],hole=.6)
fig.update_layout(title = "Total cases  : " + str(Data_World["Confirmed"][0]),)



fig.show()

In [21]:
data_over_time = data.groupby(["ObservationDate"])[["Confirmed","Active_case","Recovered","Deaths"]].sum().reset_index().sort_values("ObservationDate",ascending=True).reset_index(drop=True)
#sort_values(self, by, axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last', ignore_index=False)

In [22]:
data_over_time.head()

,ObservationDate,Confirmed,Active_case,Recovered,Deaths
0,01/22/2020,555,510,28,17
1,01/23/2020,653,605,30,18
2,01/24/2020,941,879,36,26
3,01/25/2020,1438,1357,39,42
4,01/26/2020,2118,2010,52,56


In [23]:
fig = go.Figure()

fig.add_trace(go.Scatter(y=data_over_time['Confirmed'],x=data_over_time.index, mode="lines" ,name="Confirmed cases"))

fig.update_layout(title="Cases over time" , template = "plotly_dark" , xaxis_title="Days",
    yaxis_title="Confirmed Cases",)


fig.show()

In [26]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=data_over_time.index, y=data_over_time['Active_case'],
                    mode='lines',marker_color='red',
                    name='Active cases',line=dict( dash='dot')))

fig.update_layout(
    title='Evolution of Active cases over time in the world',
        template='plotly_dark',
      yaxis_title="Active cases",
    xaxis_title="Days",

)

fig.show()
